In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
# First, enter the following code to download a Postgres driver that will allow Spark to interact with Postgres:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-06 04:37:46--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-08-06 04:37:47 (6.59 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final_data").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://tq2cutebucket.s3.us-west-1.amazonaws.com/big_main.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("big_main.csv"), sep=",", header=True, inferSchema=True)
df.show()

+------------+--------------------+--------------------+--------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+--------------+--------------+----------+
|  MLS_Number|      Street_Address|                City|Zip_Code|SqFtTotal|Lot_Size|Age|BathsTotal|BedsTotal|BathsFull|BathsHalf|DOM|Year_Sold|Year_List|List_Price|Sale_Price|  Listing_Date|     Sale_Date|Year_Built|
+------------+--------------------+--------------------+--------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+--------------+--------------+----------+
|  ML81845207|     5 Faxon Forest |            Atherton|   94027|     9587| 43791.0| 14|         7|        5|        5|        2|  5|     2021|     2021|  21995000|  23050000|5/21/2021 0:00|5/26/2021 0:00|      2007|
| TM21-737738|     807 Cinthia ST |       Beverly Hills|   90210|    13300| 30204.0|  3|        12|        7|        8|        4|  1

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://tq2cutebucket.s3.us-west-1.amazonaws.com/county_zipcode.csv"
spark.sparkContext.addFile(url)
county_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("county_zipcode.csv"), sep=",", header=True, inferSchema=True)
county_df.show()
county_df.count()

+--------+--------------+
|zip_code|        county|
+--------+--------------+
|   94063|     San Mateo|
|   92225|     Riverside|
|   92273|      Imperial|
|   92651|        Orange|
|   92801|        Orange|
|   92865|        Orange|
|   92352|San Bernardino|
|   92250|      Imperial|
|   92807|        Orange|
|   92627|        Orange|
|   91935|     San Diego|
|   91773|   Los Angeles|
|   91107|   Los Angeles|
|   93283|          Kern|
|   92866|        Orange|
|   94528|  Contra Costa|
|   95076|    Santa Cruz|
|   90010|   Los Angeles|
|   91905|     San Diego|
|   91016|   Los Angeles|
+--------+--------------+
only showing top 20 rows



51011

In [6]:
county_df = county_df.withColumnRenamed("zip_code","zipcode_b")
county_df.show()

+---------+--------------+
|zipcode_b|        county|
+---------+--------------+
|    94063|     San Mateo|
|    92225|     Riverside|
|    92273|      Imperial|
|    92651|        Orange|
|    92801|        Orange|
|    92865|        Orange|
|    92352|San Bernardino|
|    92250|      Imperial|
|    92807|        Orange|
|    92627|        Orange|
|    91935|     San Diego|
|    91773|   Los Angeles|
|    91107|   Los Angeles|
|    93283|          Kern|
|    92866|        Orange|
|    94528|  Contra Costa|
|    95076|    Santa Cruz|
|    90010|   Los Angeles|
|    91905|     San Diego|
|    91016|   Los Angeles|
+---------+--------------+
only showing top 20 rows



In [7]:
# Join the two DataFrame
# empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"fullouter") 
joined_df = df.join(county_df, df.Zip_Code == county_df.zipcode_b, how="inner")
joined_df.count()

124729

In [8]:
# Give county name a number
from pyspark.ml.feature import StringIndexer

# create object of StringIndexer class and specify input and output column
SI_county = StringIndexer(inputCol='county', outputCol='County_Index')

# transform the data
joined_df = SI_county.fit(joined_df).transform(joined_df)

# view the transformed data
joined_df.select('county', 'County_Index').show(10)

joined_df.show()

+------+------------+
|county|County_Index|
+------+------------+
|Orange|         2.0|
|Orange|         2.0|
|Orange|         2.0|
|Orange|         2.0|
|Orange|         2.0|
|Orange|         2.0|
|Orange|         2.0|
|Orange|         2.0|
|Orange|         2.0|
|Orange|         2.0|
+------+------------+
only showing top 10 rows

+------------+--------------------+--------------+--------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+--------------+--------------+----------+---------+--------------+------------+
|  MLS_Number|      Street_Address|          City|Zip_Code|SqFtTotal|Lot_Size|Age|BathsTotal|BedsTotal|BathsFull|BathsHalf|DOM|Year_Sold|Year_List|List_Price|Sale_Price|  Listing_Date|     Sale_Date|Year_Built|zipcode_b|        county|County_Index|
+------------+--------------------+--------------+--------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------

In [9]:
modify_df = joined_df.select('MLS_Number','Street_Address','City','Zip_Code','county','County_Index','SqFtTotal','Lot_Size','Age','BathsTotal','BedsTotal','BathsFull','BathsHalf','DOM','Year_List','Year_Sold','List_Price','Sale_Price','Listing_Date','Sale_Date','Year_Built')
modify_df.show(5)

+------------+--------------------+------------+--------+------+------------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+--------------+--------------+----------+
|  MLS_Number|      Street_Address|        City|Zip_Code|county|County_Index|SqFtTotal|Lot_Size|Age|BathsTotal|BedsTotal|BathsFull|BathsHalf|DOM|Year_List|Year_Sold|List_Price|Sale_Price|  Listing_Date|     Sale_Date|Year_Built|
+------------+--------------------+------------+--------+------+------------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+--------------+--------------+----------+
|CRLG21108548|1884 Upper Rim Ro...|Laguna Beach|   92651|Orange|         2.0|     1647|  6400.0| 59|         3|        2|        3|        0| 22|     2021|     2021|   1695000|   1800100| 5/3/2021 0:00|5/25/2021 0:00|      1962|
|CROC21106189|    31642 Jewel AVE |Laguna Beach|   92651|Orange|         2.0|     16

In [10]:
# import sql function pyspark
import pyspark.sql.functions as f

# null values in each column
data_agg = modify_df.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in modify_df.columns])
data_agg.show(5)

+----------+--------------+----+--------+------+------------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+------------+---------+----------+
|MLS_Number|Street_Address|City|Zip_Code|county|County_Index|SqFtTotal|Lot_Size|Age|BathsTotal|BedsTotal|BathsFull|BathsHalf|DOM|Year_List|Year_Sold|List_Price|Sale_Price|Listing_Date|Sale_Date|Year_Built|
+----------+--------------+----+--------+------+------------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+------------+---------+----------+
|         0|             0|1276|       0|     0|           0|        0|       0|  0|         0|        0|        0|        0|  0|        0|        0|         0|         0|           0|        0|         0|
+----------+--------------+----+--------+------+------------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+-----

In [11]:
# Drop null values
final_df = modify_df.dropDuplicates(['MLS_Number','Street_Address','City','Zip_Code','county','County_Index','SqFtTotal','Lot_Size','Age','BathsTotal','BedsTotal','BathsFull','BathsHalf','DOM','Year_List','Year_Sold','List_Price','Sale_Price','Listing_Date','Sale_Date','Year_Built'])
final_df.count()

4301

In [12]:
# Find out rows of null values
final_df.filter("City IS NULL").show()

+------------+--------------------+----+--------+--------------+------------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+--------------+--------------+----------+
|  MLS_Number|      Street_Address|City|Zip_Code|        county|County_Index|SqFtTotal|Lot_Size|Age|BathsTotal|BedsTotal|BathsFull|BathsHalf|DOM|Year_List|Year_Sold|List_Price|Sale_Price|  Listing_Date|     Sale_Date|Year_Built|
+------------+--------------------+----+--------+--------------+------------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+--------------+--------------+----------+
|CRIV21105771|24050 Canyon Lake...|null|   92587|     Riverside|         1.0|     2277|  7405.0| 32|         3|        3|        3|        0| 37|     2021|     2021|    725000|    740000|4/18/2021 0:00|5/25/2021 0:00|      1989|
|CRSW21111671|29941 Gulf Stream...|null|   92587|     Riverside|         1.0|     15

In [13]:
# clean up the column City with null values
# drop rows with id 4
new_final_df = final_df.filter(final_df.City!='null')
new_final_df.count()

4257

In [14]:
#Get all the Lot Size are greater than 800 in a new DataFrame.
new_final_df = new_final_df[final_df["Lot_Size"] >= 800]
new_final_df.count()

4225

In [15]:
# null values in each column
data_agg = new_final_df.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in new_final_df.columns])
data_agg.show()

+----------+--------------+----+--------+------+------------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+------------+---------+----------+
|MLS_Number|Street_Address|City|Zip_Code|county|County_Index|SqFtTotal|Lot_Size|Age|BathsTotal|BedsTotal|BathsFull|BathsHalf|DOM|Year_List|Year_Sold|List_Price|Sale_Price|Listing_Date|Sale_Date|Year_Built|
+----------+--------------+----+--------+------+------------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+------------+---------+----------+
|         0|             0|   0|       0|     0|           0|        0|       0|  0|         0|        0|        0|        0|  0|        0|        0|         0|         0|           0|        0|         0|
+----------+--------------+----+--------+------+------------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+-----

**Yeah** Data is cleaned

In [16]:
#show the clean data
new_final_df.show()

+------------+--------------------+---------------+--------+------------+------------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+--------------+--------------+----------+
|  MLS_Number|      Street_Address|           City|Zip_Code|      county|County_Index|SqFtTotal|Lot_Size|Age|BathsTotal|BedsTotal|BathsFull|BathsHalf|DOM|Year_List|Year_Sold|List_Price|Sale_Price|  Listing_Date|     Sale_Date|Year_Built|
+------------+--------------------+---------------+--------+------------+------------+---------+--------+---+----------+---------+---------+---------+---+---------+---------+----------+----------+--------------+--------------+----------+
|  ML81844669|1523 San Andreas ...|       San Jose|   95118| Santa Clara|         7.0|     1144|  5822.0| 67|         2|        3|        2|        0|  8|     2021|     2021|   1199000|   1335555|5/19/2021 0:00|5/27/2021 0:00|      1954|
|CRSN21105110|     4255 Stable LN |          Chi

**Yeah** Data is cleaned

In [17]:
new_final_df.printSchema()

root
 |-- MLS_Number: string (nullable = true)
 |-- Street_Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zip_Code: string (nullable = true)
 |-- county: string (nullable = true)
 |-- County_Index: double (nullable = false)
 |-- SqFtTotal: integer (nullable = true)
 |-- Lot_Size: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- BathsTotal: integer (nullable = true)
 |-- BedsTotal: integer (nullable = true)
 |-- BathsFull: integer (nullable = true)
 |-- BathsHalf: integer (nullable = true)
 |-- DOM: integer (nullable = true)
 |-- Year_List: integer (nullable = true)
 |-- Year_Sold: integer (nullable = true)
 |-- List_Price: integer (nullable = true)
 |-- Sale_Price: integer (nullable = true)
 |-- Listing_Date: string (nullable = true)
 |-- Sale_Date: string (nullable = true)
 |-- Year_Built: integer (nullable = true)



In [18]:
# Transpose to make it easy to read by a human being
# But it takes lots of times
# new_final_df.describe().toPandas().transpose()

In [19]:
# Make sure the df has to be pyspark before it is written
type(new_final_df)

pyspark.sql.dataframe.DataFrame

In [20]:
# Create house dataframe to show only the characteristics of the houses
house_df = new_final_df.select('MLS_Number','Street_Address','City','Zip_Code','county','County_Index','SqFtTotal','Lot_Size','BedsTotal','BathsTotal','BathsFull','BathsHalf','Year_Built',"Sale_Price")
house_df.show(5)
house_df.count()

+------------+--------------------+----------+--------+------------+------------+---------+--------+---------+----------+---------+---------+----------+----------+
|  MLS_Number|      Street_Address|      City|Zip_Code|      county|County_Index|SqFtTotal|Lot_Size|BedsTotal|BathsTotal|BathsFull|BathsHalf|Year_Built|Sale_Price|
+------------+--------------------+----------+--------+------------+------------+---------+--------+---------+----------+---------+---------+----------+----------+
|  ML81844669|1523 San Andreas ...|  San Jose|   95118| Santa Clara|         7.0|     1144|  5822.0|        3|         2|        2|        0|      1954|   1335555|
|CRSN21105110|     4255 Stable LN |     Chico|   95973|       Butte|        11.0|     1729| 34412.0|        3|         2|        2|        0|      1980|    580000|
|CRPW21106375| 2717 E Palmyra AVE |    Orange|   92869|      Orange|         2.0|     1814|  7850.0|        3|         2|        2|        0|      1963|    885000|
| ME221055681|  

4225

In [21]:
# Create sales dataframe to show information related to sales and listing sales
sale_df = new_final_df.select('MLS_Number','county','County_Index','SqFtTotal','Lot_Size', 'BedsTotal','BathsTotal','BathsFull','BathsHalf',"Age",'Year_Built','DOM','Year_Sold','Year_List','Sale_Date','List_Price','Sale_Price')
sale_df.show(8)
sale_df.count()

+------------+------------+------------+---------+--------+---------+----------+---------+---------+---+----------+---+---------+---------+--------------+----------+----------+
|  MLS_Number|      county|County_Index|SqFtTotal|Lot_Size|BedsTotal|BathsTotal|BathsFull|BathsHalf|Age|Year_Built|DOM|Year_Sold|Year_List|     Sale_Date|List_Price|Sale_Price|
+------------+------------+------------+---------+--------+---------+----------+---------+---------+---+----------+---+---------+---------+--------------+----------+----------+
|  ML81844669| Santa Clara|         7.0|     1144|  5822.0|        3|         2|        2|        0| 67|      1954|  8|     2021|     2021|5/27/2021 0:00|   1199000|   1335555|
|CRSN21105110|       Butte|        11.0|     1729| 34412.0|        3|         2|        2|        0| 41|      1980|  6|     2021|     2021|5/23/2021 0:00|    569900|    580000|
|CRPW21106375|      Orange|         2.0|     1814|  7850.0|        3|         2|        2|        0| 58|      1963|

4225

In [22]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cvmp56um73d3.us-west-1.rds.amazonaws.com:5432/Final_Project_db"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [23]:
# Write DataFrame to house table in RDS
house_df.write.jdbc(url=jdbc_url, table='house_data', mode=mode, properties=config)

In [24]:
# Write dataframe to sales table in RDS
sale_df.write.jdbc(url=jdbc_url, table='sale_data', mode=mode, properties=config)

In [25]:
# Write dataframe to final_data table in RDS
new_final_df.write.jdbc(url=jdbc_url, table='final_data', mode=mode, properties=config)